In [1]:
import imutils
import dlib
import cv2
from scipy.spatial import distance
from imutils import face_utils

In [2]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [3]:
thresh = 0.2
frame_check = 10
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("C:/Users/mayank singh/Desktop/Opencv/shape_predictor_68_face_landmarks.dat")

In [13]:

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

cap=cv2.VideoCapture(0)
flag=0

while True:
    ret, frame=cap.read()
    frame= cv2.flip(frame,1)
   # frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 225,255), 2)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 225, 255), 2)

        if ear < thresh:
            flag += 1
            #print (flag)
            if flag >= frame_check:
                cv2.putText(frame, "****************ALERT!****************", (10, 40),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.putText(frame, "****************ALERT!****************", (10,450),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                
        else:
            flag = 0

    cv2.imshow("Frame",frame)
    if cv2.waitKey(1)==13:
        break
    
cap.release()
cv2.destroyAllWindows()
